In [1]:
import numpy as np
import pandas as pd
import math
import os
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.activations import sigmoid
from keras.models import Model ,load_model
from keras.layers import Input, Dense, ConvLSTM2D, Conv2D, Conv1D, MaxPooling2D, Layer, GlobalAveragePooling2D, Reshape, Flatten, BatchNormalization, Bidirectional
from keras.regularizers import L2
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers.legacy import Adam
from keras.saving import register_keras_serializable

from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import r2_score

2024-06-15 06:45:17.744165: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 06:45:17.763894: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 06:45:17.763912: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 06:45:17.763927: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-15 06:45:17.767806: I tensorflow/core/platform/cpu_feature_g

In [2]:
# dirs
DATA_DIR = "./load.csv"
RESULT_DIR = "./result/"
TEST_PLOT_DIR = "./plots/"
load_col = 'out.site_energy.total.energy_consumption.kwh'
# MWh

In [3]:
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

if not os.path.exists(TEST_PLOT_DIR):
    os.makedirs(TEST_PLOT_DIR)

In [4]:
data = pd.read_csv(DATA_DIR)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data[load_col] = data[load_col] * 4 / 1e3


In [5]:
# scaler
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data[load_col].to_numpy().reshape(-1, 1))
data[load_col] = data_scaled

In [6]:
SEED = 42
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

In [7]:
data.describe()

,upgrade,timestamp,models_used,floor_area_represented,out.district_cooling.cooling.energy_consumption.kwh,out.district_heating.heating.energy_consumption.kwh,out.district_heating.water_systems.energy_consumption.kwh,out.electricity.cooling.energy_consumption.kwh,out.electricity.exterior_lighting.energy_consumption.kwh,out.electricity.fans.energy_consumption.kwh,...,out.electricity.total.energy_consumption.kwh.savings,out.natural_gas.total.energy_consumption.kwh.savings,out.district_heating.cooling.energy_consumption.kwh.savings,out.natural_gas.cooling.energy_consumption.kwh.savings,out.other_fuel.cooling.energy_consumption.kwh.savings,out.other_fuel.heating.energy_consumption.kwh.savings,out.other_fuel.interior_equipment.energy_consumption.kwh.savings,out.other_fuel.total.energy_consumption.kwh.savings,out.other_fuel.water_systems.energy_consumption.kwh.savings,out.site_energy.total.energy_consumption.kwh.savings
count,35040.0,35040,35040.0,3.504000e+04,35040.000000,35040.0,35040.0,35040.000000,35040.000000,35040.000000,...,35040.000000,35040.000000,35040.0,35040.0,35040.0,35040.0,35040.0,35040.0,35040.0,35040.000000
mean,18.0,2018-07-02 12:07:30,535.0,2.933235e+08,351.077590,0.0,0.0,10702.802560,3606.591142,20721.475338,...,4979.811962,23848.451624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28859.420341
min,18.0,2018-01-01 00:15:00,535.0,2.933235e+08,0.000000,0.0,0.0,326.378723,0.000000,16646.463233,...,-28508.867274,262.091277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3108.391514
25%,18.0,2018-04-02 06:11:15,535.0,2.933235e+08,56.488275,0.0,0.0,2706.386049,0.000000,19725.608356,...,-529.918116,8719.805585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17223.696432
50%,18.0,2018-07-02 12:07:30,535.0,2.933235e+08,259.402965,0.0,0.0,7965.151518,3546.633502,20458.531255,...,2273.919309,18191.504378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23244.709423
75%,18.0,2018-10-01 18:03:45,535.0,2.933235e+08,574.228447,0.0,0.0,15479.805245,7127.583771,21985.597167,...,9286.504567,32494.864830,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35158.530405
max,18.0,2019-01-01 00:00:00,535.0,2.933235e+08,1559.617664,0.0,0.0,55401.017935,7361.098134,24359.717506,...,40740.822629,159204.018407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,136399.915739
std,0.0,NaN,0.0,5.960550e-08,331.710202,0.0,0.0,9898.426784,3564.157927,1548.734876,...,8136.228348,20711.247693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17025.161426


In [8]:
class TimeSeriesImageCoder():
    def __init__(
            self,
        X: pd.DataFrame,
        n_predict: int,
        height: int,
        width: int,
        n_days: int,
        n_window_shift: str
    ) -> None:
        self.X = X
        self.h = height
        self.m = width
        self.d_b = n_days
        self.shift = n_window_shift
        self.n_predict = n_predict
        self.Lb = self.h * self.m
        self.Ls = math.ceil(self.n_predict / self.m) * self.m
        self.timestamps = self.generate_timestamps()
        print(f"Lb: {self.Lb}")
        print(f"Ls: {self.Ls}")

    def generate_timestamps(self):
        start = self.X['timestamp'].min() + DateOffset(days=self.d_b)
        end = self.X['timestamp'].max() - DateOffset(minutes=96*15)
        timestamps = pd.date_range(start=start, end=end, freq=self.shift)
        return timestamps
    
    def __make_it_symmetric_3d(self, sets_3d):
        symmetry_training_sets = []
        for slice_2d in np.array(sets_3d):
            reversed_slice_2d = slice_2d[::-1]
            combined_slice_2d = np.concatenate((slice_2d, reversed_slice_2d), axis=0)
            symmetry_training_sets.append(combined_slice_2d)
        return np.array(symmetry_training_sets)
    
    def __make_it_symmetric_2d(self, sets_2d):
        reversed_slice_2d = sets_2d[::-1]
        combined_slice_2d = np.concatenate((sets_2d, reversed_slice_2d), axis=0)
        return np.array(combined_slice_2d)
    

    def encode_b(self):
        training_sets = []
        target_sets = []
        self.X_timeseries_flatten = []
        self.X_timestamp = []
        self.y_timestamp = []
        for steps in self.timestamps:
            training_start_b = steps - DateOffset(days=self.d_b-1, hours=23, minutes=45)
            training_end = steps
            target_start = training_end + DateOffset(minutes=15)
            target_end = steps + DateOffset(minutes=(self.n_predict)*15)
            training_data = self.X[(self.X['timestamp'] >= training_start_b) & (self.X['timestamp'] <= training_end)]
            target_data = self.X[(self.X['timestamp'] >= target_start) & (self.X['timestamp'] <= target_end)]
            if not training_data.empty and not target_data.empty:
                self.X_timeseries_flatten.append(training_data[load_col])
                self.X_timestamp.append(training_data['timestamp'])
                self.y_timestamp.append(target_data['timestamp'])
                training_reshaped = np.array(training_data[load_col]).reshape(self.d_b, self.h, self.m)
                # symmetric_3d = self.__make_it_symmetric_3d(training_reshaped)
                training_sets.append(training_reshaped)
                target_reshaped = np.array(target_data[load_col]).reshape(math.ceil(self.n_predict/self.m), min(self.n_predict, self.m))
                # symmetric_2d = self.__make_it_symmetric_2d(target_reshaped)
                target_sets.append(target_reshaped.flatten())
        training_sets = np.array(training_sets)
        target_sets = np.array(target_sets)

        self.X_timeseries_flatten = np.array(self.X_timeseries_flatten)
        self.X_timestamp = np.array(self.X_timestamp)
        self.y_timestamp = np.array(self.y_timestamp)
        return training_sets, target_sets
    
    def encode(self):
        training_sets_b, target_sets = self.encode_b()
        # training_sets_s = self.encode_s()
        training_sets_b = np.transpose(training_sets_b, (0, 2, 3, 1))
        # training_sets_s = np.transpose(training_sets_s, (0, 2, 3, 1))
        return training_sets_b, target_sets

    """Use predictions from previous steps to add new inputs to rolling predictions"""
    def image_shift(self, original_input, new_input):
        input = np.transpose(original_input, (0, 3, 1, 2))
        output = []
        output.append(input[0])
        output = np.array(output).flatten()
        output = np.concatenate([output, new_input.flatten()], axis=0)[-len(output):]
        image = output.reshape(self.d_b, self.h, self.m)
        image = image.reshape(1, *image.shape)
        image = np.transpose(image, (0, 2, 3, 1))
        return image


In [9]:
encoder = TimeSeriesImageCoder(
    X=data,
    n_predict=96,
    height=8,
    width=12,
    n_days=3,
    n_window_shift="15min"
)
encoded_Xb, encoded_y = encoder.encode()
X_timeseries = np.copy(encoder.X_timeseries_flatten)
X_timestamp = np.copy(encoder.X_timestamp)
y_timestamp = np.copy(encoder.y_timestamp)

Lb: 96
Ls: 96


In [10]:
# encoder_5 = TimeSeriesImageCoder(
#     X=data,
#     n_predict=96,
#     height=8,
#     width=12,
#     n_days=5,
#     n_window_shift="15min"
# )
# encoded_Xb_5, encoded_y_5 = encoder_5.encode()

In [11]:
# encoder_7 = TimeSeriesImageCoder(
#     X=data,
#     n_predict=96,
#     height=8,
#     width=12,
#     n_days=7,
#     n_window_shift="15min"
# )
# encoded_Xb_7, encoded_y_7 = encoder_7.encode()

In [12]:
print(encoded_Xb.shape)
print(encoded_y.shape)

print(X_timeseries.shape)
print(X_timestamp.shape)
print(y_timestamp.shape)

MONTH_TIME_STEP = math.floor(encoder.timestamps.shape[0] / 24)
X_test_b = []
y_test = []
X_test_b_flatten = []
X_test_b_timestamp = []
y_test_timestamp = []

for i in range(0, 24):
    start = (i+1)*MONTH_TIME_STEP-(192*(i+1))
    end = (i+1)*MONTH_TIME_STEP-(192*i)
    X_test_b.append(encoded_Xb[start:end])
    y_test.append(encoded_y[start:end])
    X_test_b_flatten.append(X_timeseries[start:end])
    X_test_b_timestamp.append(X_timestamp[start:end])
    y_test_timestamp.append(y_timestamp[start:end])


    encoded_Xb = np.concatenate([encoded_Xb[:start], encoded_Xb[end:]])
    encoded_y = np.concatenate([encoded_y[:start], encoded_y[end:]])
    X_timeseries = np.concatenate([X_timeseries[:start], X_timeseries[end:]])
    X_timestamp = np.concatenate([X_timestamp[:start], X_timestamp[end:]])
    y_timestamp = np.concatenate([y_timestamp[:start], y_timestamp[end:]])
 
X_test_b = np.concatenate([i for i in X_test_b])
y_test = np.concatenate([i for i in y_test])
X_test_b_flatten = np.concatenate([i for i in X_test_b_flatten])
X_test_b_timestamp = np.concatenate([i for i in X_test_b_timestamp])
y_test_timestamp = np.concatenate([i for i in y_test_timestamp])

X_train_b = encoded_Xb
y_train = encoded_y

print(np.array(X_train_b).shape)
print(np.array(X_test_b).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)
print(X_test_b_flatten.shape)
print(X_test_b_timestamp.shape)
print(y_test_timestamp.shape)

(34656, 8, 12, 3)
(34656, 96)
(34656, 288)
(34656, 288)
(34656, 96)
(30048, 8, 12, 3)
(4608, 8, 12, 3)
(30048, 96)
(4608, 96)
(4608, 288)
(4608, 288)
(4608, 96)


In [13]:
def cal_err(y_test, y_pred):
    y_test_inv = scaler.inverse_transform(y_test)
    y_pred_inv = scaler.inverse_transform(y_pred)


    mse_inv = mean_squared_error(y_test_inv, y_pred_inv)
    rmse_inv = math.sqrt(mse_inv)
    mae_inv = mean_absolute_error(y_test_inv, y_pred_inv)
    mape_inv = mean_absolute_percentage_error(y_test_inv, y_pred_inv)
    r2_inv = r2_score(y_test_inv, y_pred_inv)

    print("-" * 86)
    print(f'mse_inv: {mse_inv:.4f}')
    print(f'rmse_inv: {rmse_inv:.4f}')
    print(f'mae_inv: {mae_inv:.4f}')
    print(f'mape_inv: {mape_inv: .4f}')
    print(f'r2_inv: {r2_inv:.4f}')
    print("-" * 86)

def cal_err_per_step(y_test, y_pred):
    n_steps = y_pred.shape[1]  # 預測步數

    mse = []
    rmse = []
    mae = []
    mape = []

    for i in range(n_steps):
        true_step = y_pred[:, i]
        predicted_step = y_test[:, i]

        mse_step = mean_squared_error(true_step, predicted_step)
        rmse_step = np.sqrt(mse_step)
        mae_step = mean_absolute_error(true_step, predicted_step)
        mape_step = mean_absolute_percentage_error(true_step, predicted_step) * 100

        mse.append(mse_step)
        rmse.append(rmse_step)
        mae.append(mae_step)
        mape.append(mape_step)

    return mse, rmse, mae, mape


In [14]:
proposed = pd.read_csv("./result/proposed_96.csv", index_col=0).to_numpy()
PhaCIA_TCN = pd.read_csv("./result/PhaCIA_TCN.csv", index_col=0).to_numpy()
Image_CNN = pd.read_csv("./result/Image_CNN.csv", index_col=0).to_numpy()


In [15]:
cal_err(y_test, proposed)
cal_err(y_test, PhaCIA_TCN)
cal_err(y_test, Image_CNN)

results = {
    "proposed": cal_err_per_step(y_test, proposed),
    "PhaCIA_TCN": cal_err_per_step(y_test, PhaCIA_TCN),
    "Image_CNN": cal_err_per_step(y_test, Image_CNN)
}

--------------------------------------------------------------------------------------
mse_inv: 203.5235
rmse_inv: 14.2662
mae_inv: 10.2907
mape_inv:  0.0260
r2_inv: 0.9668
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
mse_inv: 8543.9053
rmse_inv: 92.4332
mae_inv: 63.3535
mape_inv:  0.1585
r2_inv: -0.3920
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
mse_inv: 1225.1791
rmse_inv: 35.0026
mae_inv: 25.2634
mape_inv:  0.0625
r2_inv: 0.7998
--------------------------------------------------------------------------------------


In [16]:
# plt.rcParams['font.family'] = 'Times New Roman'
# plt.figure(figsize=(12, 7.5))
# plt.title('Comparison of Different Model MAPE for Each Step', fontsize=18)
# plt.plot(results["proposed"][-1], label="Proposed")
# plt.plot(results["PhaCIA_TCN"][-1], label="PhaCIA-TCNs [48]")
# plt.plot(results["Image_CNN"][-1], label="STI-CNN [49]")
# plt.legend(fontsize=14)
# plt.xlabel("Steps", fontsize=14)
# plt.ylabel("MAPE (%)", fontsize=14)
# plt.tight_layout()
# plt.savefig("Different_Model_MAPE.png", dpi=1300)
# plt.show()

In [18]:
pred_data1 = scaler.inverse_transform(proposed)
pred_data2 = scaler.inverse_transform(Image_CNN)
actual_data = scaler.inverse_transform(y_test)
previous_data = scaler.inverse_transform(X_test_b_flatten)

plt.rcParams['font.family'] = 'Times New Roman'
for i in range(actual_data.shape[0]):
    if i in [187, 231, 628, 1145, 1508, 1903, 2491, 2628, 3176, 3453, 4029, 4223, 4511]:
        history = None
        history_timestamp = None
        index = i % 192
        if index < 47:
            history = pred_data2[i-index:i+1, 0]
            history_timestamp = y_test_timestamp[i-index:i+1, 0]
        else:
            history = pred_data2[i-48:i+1, 0]
            history_timestamp = y_test_timestamp[i-48:i+1, 0]
            
        plt.figure(figsize=(12, 6))
        X1 = np.concatenate((X_test_b_timestamp[i][-48:], y_test_timestamp[i]))
        y1 = np.concatenate((previous_data[i][-48:], actual_data[i]))
        X2 = y_test_timestamp[i]
        y_p1 = pred_data1[i]
        y_p2 = pred_data2[i]
        y_a = actual_data[i]
        Xh = np.full(700, X1[len(X1)-96])
        yh = np.arange(0, 700, 1)
        plt.title(f"Day Ahead Forecasting Starting From {pd.to_datetime(X2[0]).strftime('%m/%d %H:%M')}", fontsize=18)
        plt.plot(X1, y1, '.-', label='Actual', color='#6eb5c0')
        plt.plot(X2, y_p1, '.-', label='Predicted', color='#1167b1')
        # plt.plot(X2, y_p2, '.-', label='Rolling Forecasting', color='#03264c')
        plt.plot(history_timestamp, history, '--', label='Rolling Forecasting History', color='#989898')
        plt.plot(Xh, yh, color='#4863a0', alpha=0.5)
        plt.ylim(0, 700)
        plt.xlabel('Time step', fontsize=14)
        plt.ylabel('Usage (kWh)', fontsize=14)
        plt.legend(loc='lower left', bbox_to_anchor=(1, 0), fontsize=11)
        plt.tight_layout()
        plt.savefig(TEST_PLOT_DIR+f"Time_Series_{i+1}.png", dpi=1300)
        plt.close()


findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times Ne

findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times New Roman' not found.
findfont: Font family 'Times Ne